# Problem 1

In [3]:
install.packages(c("boot","msm","dplyr")) #We start by installing and loading necessary packages
library(boot)
library(msm)
library(dplyr)

Installing packages into 'C:/Users/Honza/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'dplyr' is in use and will not be installed"

package 'boot' successfully unpacked and MD5 sums checked
package 'msm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Honza\AppData\Local\Temp\RtmpqeyR0Q\downloaded_packages


Warning message:
"package 'boot' was built under R version 3.6.3"Warning message:
"package 'msm' was built under R version 3.6.3"
Attaching package: 'msm'

The following object is masked from 'package:boot':

    cav



In [4]:
library(boot)
data(city)
data <- city
data
#data <- data(city) #we load the data
#data #let's take a look, dataset is so small that we can view it as a whole

u,x
138,143
93,104
61,69
179,260
48,75
37,63
29,50
23,48
30,111
2,50


## a) Draw the dependence between $u_i$ and $x_i$

## b)

In [10]:
R = mean(data$x)/mean(data$u)
R
sample_n(data, nrow(data), replace = TRUE)

[1] 1.520312

,u,x
2...1,93,104
5,48,75
7,29,50
10,2,50
8...5,23,48
4,179,260
2...7,93,104
8...8,23,48
2...9,93,104
6,37,63


# c)

In [7]:
# brute force bootstraping
Boot    = 9999
x.boot = rep(0, Boot)
u.boot = rep(0, Boot)

set.seed(42)
for(i in 1:Boot) {
  sample_data <- sample_n(data, nrow(data), replace = TRUE)
  x.boot <- mean(sample_data$x)
  u.boot <- mean(sample_data$u)
}

In [9]:
r <- x.boot/u.boot
R_bf = mean(r) #bf stands for brute force
sprintf("Bootstrap estimate of R: %.4f", R_bf)

[1] "Bootstrap estimate of R: 1.5546"

In [ ]:
z <- c(1:10)
set.seed(42)
for(i in 1:Boot) {
  data_sample = sample_n()   
}

# Problem 3

In [ ]:
install.packages(c("readr","tseries","gmm")) #necessary packages
library(readr) #so that we can load the data
library(tseries)
library(gmm)

In [ ]:
data3 <- read_csv("hw2_data.csv")
head(data3)

In [ ]:
install.packages("rlang")